# Import necessary libraries

In [1]:
# General
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle
import time
import copy
import pandas as pd


# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms


# PySyft
import syft as sy
from syft.frameworks.torch.fl import utils
from syft.workers.websocket_client import WebsocketClientWorker

# Pre-processing the Data

In [2]:
# Set the image size Y where Y represents YxY 
IMG_SIZE = 64
BATCH_SIZE = 100
LR = 0.001

In [3]:
train = datasets.MNIST(r"/media/wilfredo/Willie931GB/EURECOM_SLU_Linux/II_SEMESTER/SLU/PAPER_KDD2022/EXPERIMENTS/PySyft/Datasets/MNIST_64", 
                      train = True, download = True, 
                      transform = transforms.Compose([transforms.Resize(IMG_SIZE),
                                                      transforms.ToTensor()]))

test = datasets.MNIST(r"/media/wilfredo/Willie931GB/EURECOM_SLU_Linux/II_SEMESTER/SLU/PAPER_KDD2022/EXPERIMENTS/PySyft/Datasets/MNIST_64", 
                      train = False, download = True, 
                      transform = transforms.Compose([transforms.Resize(IMG_SIZE),
                                                      transforms.ToTensor()]))

In [4]:
# Load the data from the file it was saved in. Take the ENTIRE dataset!
training_data = torch.utils.data.DataLoader(train, batch_size = int(len(train)/2), shuffle = True)
test_data = torch.utils.data.DataLoader(test, batch_size = int(len(test)/2), shuffle = True)

# Create the CNN (based on VGG11)
Source: Page 3/14, Table 1, Configuration A, https://arxiv.org/pdf/1409.1556.pdf

## Model on clients (small portion)

In [5]:
class Net_client(nn.Module):
    def __init__(self):
        super().__init__()
        # Define your first convolutional layer: input = 1, output = 32 convolutional features, kernel size = 5
        # Remember that kernel = 5 means that the "window" used to scan for features will be 5x5
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)

    # Function defining only one part of the forward pass (the convolution layers only). This will also write
    # the output dimensions of the conv layers to self._to_linear ONCE, and this information will then be used 
    # as the input data flattened dimensions of the next fully connected layers 
    def convs(self, x):
        # Convolutional layer 1 + activation + max_pooling
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2))
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2))
        return x
    
    # Function defining the rest of the forward pass
    def forward(self, x):
        # Run the convs layers first
        x = self.convs(x)
        return x

net_client = Net_client()

## Model on Server (big portion)

In [6]:
class Net_server(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Start from the third convolutional layer
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 3)
        
        # Run the fully connected layers. We know the input of this fc1 layer is 512, because of our previous
        # results with FL, where self.__to__linear told us this result when you run the cell that contains the 
        # NN
        self._to_linear = 1024
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)

    # Function defining only one part of the forward pass (the convolution layers only). This will also write
    # the output dimensions of the conv layers to self._to_linear ONCE, and this information will then be used 
    # as the input data flattened dimensions of the next fully connected layers 
    def convs(self, x):
        # Convolutional layer 1 + activation + max_pooling
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2))
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x
    
    # Function defining the rest of the forward pass
    def forward(self, x):
        # Run the convs layers first
        x = self.convs(x)
        # Reshape the output data from the convs to be flattened
        x = x.view(-1, self._to_linear)
        # Pass the data through the fully connected layers now
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # Pass it through the final layer
        x = self.fc3(x)
        # One final softmax function to make the output vector look nicer
        x = F.softmax(x, dim = 1)
        return x

net_server = Net_server()

In [7]:
# Take a look at our models
model_client = net_client
model_server = net_server

In [8]:
# Take a look at your model
model_client

Net_client(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
)

In [9]:
# Take a look at your model
model_server

Net_server(
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)

# Establish your loss function

In [10]:
# Set your loss function (MSE for images!)
loss_function = nn.MSELoss()

# Separate your data into data, labels, training, testing, and scale it

In [11]:
# Take the data loaded onto training_data. You NEED to iterate over it to take it, even if you
# want to take the entire thing. Make sure to convert the values to floats
X = next(iter(training_data))[0]
y_unformatted = next(iter(training_data))[1].type(torch.FloatTensor)
X_test = next(iter(test_data))[0]
y_test_unformatted = next(iter(test_data))[1].type(torch.FloatTensor)

# The two other cases in this paper use 2 dimensional labels (0, 1), not only (0)
# MNIST by default comes with labels in the format (9) instead of (9, 0). To change this:
# Create tensors with all zeros of the same size
y_unformatted_addition = torch.zeros(y_unformatted.size())
y_test_unformatted_addition = torch.zeros(y_test_unformatted.size())
# Then stack them together (0 for vertically, -1 for horizontally)
y = torch.stack((y_unformatted, y_unformatted_addition), -1)
y_test = torch.stack((y_test_unformatted, y_test_unformatted_addition), -1)


In [12]:
# Define your training data
# train_X = X[:-val_size]
# train_y = y[:-val_size]
train_X = X
train_y = y

# Define your testing (validation) data
# test_X = X[-val_size:]
# test_y = y[-val_size:]
test_X = X_test
test_y = y_test

# Split Learning

## Establish the virtual workers, their data, their NNs, and their optimizers

In [13]:
# Start the hook
hook = sy.TorchHook(torch)

# Create your virtual workers and our server
worker1 = sy.VirtualWorker(hook, id="worker1")
worker2 = sy.VirtualWorker(hook, id="worker2")
worker3 = sy.VirtualWorker(hook, id="worker3")
server = sy.VirtualWorker(hook, id="server")

# Put the WORKERS into a list for easier access later on
compute_nodes = [worker1, worker2, worker3]

In [14]:
# Split the training data for each worker
# General method is:
# train_X_workerN = train_X[int((N-1) * len(train_X)/len(compute_nodes)):int(N * len(train_X)/len(compute_nodes))].view(-1, 1, IMG_SIZE, IMG_SIZE)
train_X_worker1 = train_X[:int(len(train_X)/len(compute_nodes))].view(-1, 1, IMG_SIZE, IMG_SIZE)
train_X_worker2 = train_X[int(len(train_X)/len(compute_nodes)):int(2 * len(train_X)/len(compute_nodes))].view(-1, 1, IMG_SIZE, IMG_SIZE)
train_X_worker3 = train_X[int(2 * len(train_X)/len(compute_nodes)):int(3 * len(train_X)/len(compute_nodes))].view(-1, 1, IMG_SIZE, IMG_SIZE)

train_y_worker1 = train_y[:int(len(train_X)/len(compute_nodes))]
train_y_worker2 = train_y[int(len(train_X)/len(compute_nodes)):int(2 * len(train_X)/len(compute_nodes))]
train_y_worker3 = train_y[int(2 * len(train_X)/len(compute_nodes)):int(3 * len(train_X)/len(compute_nodes))]

In [15]:
# Clear the workers of any objects, just in case you forgot some were still there from a previous run
worker1.clear_objects()
worker2.clear_objects()
worker3.clear_objects()
server.clear_objects()

<VirtualWorker id:server #objects:0>

In [16]:
# # Establish the NN model for each worker. This is model-centric FL, so it is the same model for all workers
worker1_model = model_client.copy()
worker2_model = model_client.copy()
worker3_model = model_client.copy()
server_model = model_server.copy()
# worker_model = model_client.copy()

# Establish the optimizer for each worker
worker1_optimizer = optim.SGD(worker1_model.parameters(), lr=LR)
worker2_optimizer = optim.SGD(worker2_model.parameters(), lr=LR)
worker3_optimizer = optim.SGD(worker3_model.parameters(), lr=LR)
# worker_optimizer = optim.SGD(worker_model.parameters(), lr=LR)
server_optimizer = optim.SGD(server_model.parameters(), lr=LR)

In [17]:
# Organize the WORKER models and optimizers into lists. The server stuff must not be mixed with these
models = [worker1_model, worker2_model, worker3_model]
optimizers = [worker1_optimizer, worker2_optimizer, worker3_optimizer]

worker_collection = [[worker1, worker1_model, worker1_optimizer], 
                     [worker2, worker2_model, worker2_optimizer], 
                    [worker3, worker3_model, worker3_optimizer]]

## Training Sequence

In [18]:
def train():
    
    # We are going to train all workers, sequentialy, with all batches of their respective data.
    # AFTER the training for one is done, THEN we move on to the next. So:
    epoch_time = 0
    batch_count = 0
    batch_times = []
    total_epoch_time = 0
    
    for worker, model, optimizer in worker_collection:
        
        for i in tqdm(range(0, int(len(train_X)/len(compute_nodes)), BATCH_SIZE)):
            
            # Send the models to the worker. This step is not necessary in real life, as this is only done once. 
            # However, because the .rm_obj() method is not working to remove the batches at the end of each loop,
            # we are forced to remove ALL objects from each worker, including the models, so the batches don't
            # constantly add up, consuming more memory each time!
            model.send(worker)
            server_model.send(server)
            
            # Make sure we're working with the correct batches
            if worker is worker1:
                batch_X = train_X_worker1[i : i + BATCH_SIZE]
                batch_y = train_y_worker1[i : i + BATCH_SIZE]
            elif worker is worker2:
                batch_X = train_X_worker2[i : i + BATCH_SIZE]
                batch_y = train_y_worker2[i : i + BATCH_SIZE]
            elif worker is worker3:
                batch_X = train_X_worker3[i : i + BATCH_SIZE]
                batch_y = train_y_worker3[i : i + BATCH_SIZE]
            
            # Send ONLY the data  to each of the workers. This does not have to be done in real life, but 
            # must be done in this simulation. The LABELS are sent to the SERVER!
            batch_X = batch_X.send(worker)
            batch_y = batch_y.send(server)

            # Zero the sequence for all models on both workers and server!
            optimizer.zero_grad()
            server_optimizer.zero_grad()

            # Start FP on worker
            FP_client_start_time = time.time()
            intermediate = model(batch_X)
            FP_client_end_time = time.time() - FP_client_start_time
            
            # Split the forward pass here. Send the result of the FP up to this point from the worker to 
            # the server
            remote_intermediate = intermediate.detach().move(server).requires_grad_()

            # Complete FP on servers
            FP_server_start_time = time.time()
            pred = server_model(remote_intermediate)
            FP_server_end_time = time.time() - FP_server_start_time

            # Calculate loss on server
            BP_server_start_time = time.time()
            loss = loss_function(pred, batch_y)
            # Start BP on server
            loss.backward()
            # Update weights on server
            server_optimizer.step()
            BP_server_end_time = time.time() - BP_server_start_time
            
            # Continue BP on client
            intermediate.move(worker)
            grad_intermediate = remote_intermediate.grad.copy().move(worker)
            
            BP_client_start_time = time.time()
            intermediate.backward(grad_intermediate)
            server_optimizer.step()
            BP_client_end_time = time.time() - BP_client_start_time
            
            
            # Total batch time
            total_batch_time_worker = FP_client_end_time + FP_server_end_time + BP_server_end_time + BP_client_end_time
#             print("Total batch time for this worker = ", round(total_batch_time_worker, 4), " s")
#             print("Total batch time for all workers = ", round(total_batch_time_worker*len(compute_nodes), 4), " s")
#             total_batch_time += total_batch_time_worker
            total_epoch_time += total_batch_time_worker
    
            # Save the batch time for this batch. For simplicity, we assume the time it took
            # worker1 to do one batch is the same time it took the others to do their own
            # respective batch
            if worker is worker1:
                batch_times.append(total_batch_time_worker * len(compute_nodes))
            
            # The following steps must not be done in real life. They are only done as a memory saving measure
            # in this implementation because the .rm_obj() method does not work to remove the batches from 
            # the worker's memory after it is done
            
            # Get back the models
            model.get()
            server_model.get()
            
            # Clear the objects from the workers and servers
            worker.clear_objects()
            server.clear_objects()
            
#             batch_count += 1
#             if batch_count >= 25:
#                 break
            
    
        # OUTSIDE first for loop!        
        # Have the current worker send its model to the next worker
        if worker is worker1:
            # Make worker2's model be the finished model from worker1
            worker_collection[1][1] = worker1_model.copy()
#             worker2_model = worker1_model.copy()
        elif worker is worker2:
            # Make worker3's model be the finished model from worker2
            worker_collection[2][1] = worker2_model.copy()
#             worker3_model = worker2_model.copy()
        # Nothing after this because worker3 won't send his model to anyone!
    
    # OUTISDE BIGGER FOR LOOP  
    print("TOTAL TIME FOR THIS EPOCH = ", round(total_epoch_time, 4), " s")
#     Return the new model on the server
    return model, server_model, batch_times, total_epoch_time

## Function used for testing

In [19]:
def test(new_worker_model, new_server_model):
    
    # Calculate the accuracy
    correct = 0
    total = 0

    # Do not update your gradients while testing
    with torch.no_grad():
        print("Initiated model testing:")
        for i in tqdm(range(len(test_X))):
            
            # Put the model into evaluation mode so it does not update its gradients during this test
            new_worker_model.eval()
            new_server_model.eval()

            # Obtain the real class for the sample
            real_class = torch.argmax(test_y[i])

            # Obtain our prediction for said sample (not arg_maxed yet)
#             output = new_model_server(test_X[i].view(-1, 1, IMG_SIZE, IMG_SIZE))[0]
            output = new_server_model(new_worker_model(test_X[i].view(-1, 1, IMG_SIZE, IMG_SIZE)))[0]
            
            # Obtain our arg_maxed prediction for said sample
            predicted_class = torch.argmax(output)

            # Update counters
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy of the new model = ", round(correct/total, 3), "\n \n")

## Reset function

In [20]:
def update_models(new_worker_model, new_server_model):
    # Clear the workers of any objects, just in case you forgot some were still there from a previous run
    worker1.clear_objects()
    worker2.clear_objects()
    worker3.clear_objects()
    server.clear_objects()
    
    # Establish the NN model for each worker. This is model-centric FL, so it is the same model for all workers
    worker1_model = new_worker_model.copy()
    worker2_model = new_worker_model.copy()
    worker3_model = new_worker_model.copy()
    server_model = new_server_model.copy()

    # Establish the optimizer for each worker
    worker1_optimizer = optim.SGD(worker1_model.parameters(), lr=LR)
    worker2_optimizer = optim.SGD(worker2_model.parameters(), lr=LR)
    worker3_optimizer = optim.SGD(worker3_model.parameters(), lr=LR)
    server_optimizer = optim.SGD(server_model.parameters(), lr=LR)
    
    # Organize the WORKER models and optimizers into lists. The server stuff must not be mixed with these
    global models
    models = [worker1_model, worker2_model, worker3_model]
    global optimizers
    optimizers = [worker1_optimizer, worker2_optimizer, worker3_optimizer]
    global worker_collection
    worker_collection = [[worker1, worker1_model, worker1_optimizer], 
                         [worker2, worker2_model, worker2_optimizer], 
                        [worker3, worker3_model, worker3_optimizer]]

# RUN THE MODEL

In [21]:
# Define your number of epochs
epochs = 4
epoch_times = []

# Train all workers for the set number of epochs
for epoch in range(epochs):
    
    # Start counting the time for this epoch
    start_time = time.time()
    print(f"Epoch Number {epoch + 1}")
    
    # Train the individual models, and then obtain the federated averaged model
#     train_start_time = time.time()
    new_worker_model, new_server_model, batch_times, epoch_time = train()
#     train_total_time = time.time() - train_start_time
#     print("Total TRAIN time for epoch ", epoch, " = ", 
#           round(train_total_time/60, 2), " min")
    
    # Save the epoch times
    epoch_times.append(epoch_time)
    
    # Stop counting the time
#     epoch_total_time = time.time() - start_time
#     print('Time for this epoch', round(epoch_total_time/60, 2), 'min')
    
    # Test your new model to keep a log of how good we're doing per epoch 
    test(new_worker_model, new_server_model)
    
    # Update all models before the next epoch
    update_models(new_worker_model, new_server_model)
    
    # Save all batch times
    df_batch = pd.DataFrame(batch_times)
    df_batch.to_csv("./Batch_times/STD_MNIST_SL_epoch_" + str(epoch) + ".csv")

# OUTSIDE THE FOR LOOP
# Save the epoch times
df_epoch = pd.DataFrame(epoch_times)
df_epoch.to_csv("./Epoch_times/STD_MNIST_SL.csv")


# Clean the global namespace after run is done
%reset -f

Epoch Number 1


100%|██████████| 100/100 [05:23<00:00,  3.23s/it]


TOTAL TIME FOR THIS EPOCH =  130.7924  s
Initiated model testing:


100%|██████████| 5000/5000 [00:24<00:00, 202.59it/s]


Accuracy of the new model =  0.903 
 

Epoch Number 2


...

Accuracy of the new model =  0.903 
 

Epoch Number 4


100%|██████████| 100/100 [05:21<00:00,  3.21s/it]


TOTAL TIME FOR THIS EPOCH =  123.8991  s
Initiated model testing:


100%|██████████| 5000/5000 [00:26<00:00, 191.45it/s]


Accuracy of the new model =  0.903 
 

